In [ ]:
import pandas as pd
import numpy as np
import gc # garbage collection
from kaggle_datasets import KaggleDatasets
import re, math
import tensorflow.keras.backend as K
from tensorflow.python.keras.utils import losses_utils
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pickle
import random as r
import cv2, os
from sklearn.metrics import classification_report

In [ ]:
DEVICE = "TPU" # or "GPU"
SEED = 42 # USE DIFFERENT SEED FOR DIFFERENT STRATIFIED KFOLD
FOLDS = 5 # NUMBER OF FOLDS. USE 3, 5, OR 15 
IMG_SIZES = [384] * FOLDS
BATCH_SIZES = [32] * FOLDS
EPOCHS = [40] * FOLDS
EFF_NETS = [5] * FOLDS
WGTS = [1 / FOLDS] * FOLDS # WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
TTA = 15 # TEST TIME AUGMENTATION FACTOR
NUMOFCLASSES = 7
CLASSWEIGHTS = {0:1.28545758 , 1: 0.21338021,2:2.78349083 ,3:4.37527304 ,4:1.30183284,5:12.44099379,6:10.07545272} 

CLASSES = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']
VERBOSE = 1
DISPLAY_PLOT = True

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
clsz=7

In [ ]:

predwithtta=pd.read_csv('/kaggle/input/hame5output/HAMDiswith_TTA_submission.csv')
predwithouttta=pd.read_csv('/kaggle/input/hame5output/HAMDiswithout_TTA_submission.csv')

for fold,(idxT,idxV) in enumerate(skf.split(np.arange(FOLDS))):  
    with open("/kaggle/input/hame5output/Dis_history_fold-%i.p"%(fold+1), 'rb') as pickle_file:
        history = pickle.load(pickle_file)
#         print(history)
#         DISPLAY_PLOT=False
#         break
    #hist = pickle.load(")
    #pickle.dump(hist, open("history_fold-%i.p"%(fold+1), "wb"))
    # PLOT TRAINING
        if DISPLAY_PLOT:
            plt.figure(figsize=(15,5))
            plt.plot(np.arange(len(history['sparse_categorical_accuracy'])),history['sparse_categorical_accuracy'],'-o',label='Train sparse_categorical_accuracy',color='#ff7f0e')
            plt.plot(np.arange(len(history['sparse_categorical_accuracy'])),history['val_sparse_categorical_accuracy'],'-o',label='Val val_sparse_categorical_accuracy',color='#1f77b4')
            x = np.argmax( history['val_sparse_categorical_accuracy'] ); y = np.max( history['val_sparse_categorical_accuracy'] )
            xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
            plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max sparse_categorical_accuracy\n%.4f'%y,size=14)
            plt.ylabel('acc',size=14); plt.xlabel('Epoch',size=14)
            plt.legend(loc=2)
            plt2 = plt.gca().twinx()
            plt2.plot(np.arange(len(history['student_loss'])),history['student_loss'],'-o',label='student_loss',color='#FF0000')
            plt2.plot(np.arange(len(history['distillation_loss'])),history['distillation_loss'],'-o',label='distillation_loss',color='#2ca02c')
            plt2.plot(np.arange(len(history['distillation_loss'])),history['val_student_loss'],'-o',label='val_student_loss',color='#1f77b4')
            x = np.argmin( history['val_student_loss'] ); y = np.min( history['val_student_loss'] )
            ydist = plt.ylim()[1] - plt.ylim()[0]
            #plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss%.4f'%y,size=14)
            plt.ylabel('Loss',size=14)
            plt.title('FOLD %i '% (fold+1),size=18)
            plt.legend(loc=3)
            plt.show()
clsz = 7            
actual=pd.read_csv('/kaggle/input/isic2018task3groundtruth/ISIC2018_Task3_Test_GroundTruth.csv')
actual = actual.drop(['lesion_id','dx_type','age','localization','dataset','sex'], axis=1)
dummies = pd.get_dummies(actual['dx'])
actual = pd.concat([actual, dummies], axis=1)
actual = actual.drop(['dx'],axis=1)
actual=actual.reindex(columns=['image_id','mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc'])
actual['mel'] = actual['mel'].astype(float)
actual['nv'] = actual['nv'].astype(float)
actual['bcc'] = actual['bcc'].astype(float)
actual['akiec'] = actual['akiec'].astype(float)
actual['bkl'] = actual['bkl'].astype(float)
actual['df'] = actual['df'].astype(float)
actual['vasc'] = actual['vasc'].astype(float)
print(actual.head())


predwithtta = predwithtta.rename(columns={'image_name' :'image_id','targetA':'mel','targetB':'nv','targetC':'bcc','targetD':'akiec','targetE':'bkl','targetF':'df','targetG':'vasc'})
def set_max_to_one(row):
    #print(type(row))
    max_valuei = row.idxmax()
    row[:] = 0.0
    row[max_valuei] = 1.0
    return row

# Apply the function to each row of the DataFrame
predwithtta[['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']] = predwithtta[['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']].apply(set_max_to_one, axis=1)

# Display the modified DataFrame
print(predwithtta.head())



# Load ground truth and predicted dataframes
dfA = actual
dfB = predwithtta
dfB.columns = [dfB.columns[0]] + [col + "_y" for col in dfB.columns[1:]]

dfC = pd.merge(dfA, dfB, on='image_id')
print(dfC.head())

# Define the labels for your classification problem
labels = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']

# Create a list to store the accuracy for each label
label_acc = []
print("WIth TTA")

# Loop through the labels and calculate accuracy, precision, and recall for each label
for label in labels:
    # Calculate accuracy for the label
    accuracy = (dfC[label] == dfC[label+'_y']).mean()
    label_acc.append(accuracy)
    
    # Calculate precision, recall, and F1-score for the label using classification_report
    report = classification_report(dfC[label], dfC[label+'_y'], output_dict=True)

    
    # Print the metrics for the label
    print('Label:', label)
    print('stats',report['1.0'])
    print('---')


overall_acc=sum(label_acc)/clsz
print('overall_acc:', overall_acc)
dfC.to_csv('HAMdistE5E5TTAandgroundtruth_alpha-0-35-Notemp.csv', index=False)


predwithouttta = predwithouttta.rename(columns={'image_name' :'image_id','targetA':'mel','targetB':'nv','targetC':'bcc','targetD':'akiec','targetE':'bkl','targetF':'df','targetG':'vasc'})
def set_max_to_one(row):
    #print(type(row))
    max_valuei = row.idxmax()
    #print(row[:])
    row[:] = 0.0
    row[max_valuei] = 1.0
    return row

# Apply the function to each row of the DataFrame
predwithouttta[['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']] = predwithouttta[['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']].apply(set_max_to_one, axis=1)

# Display the modified DataFrame
print(predwithouttta.head())

# Load ground truth and predicted dataframes
dfA = actual
dfB = predwithouttta
dfB.columns = [dfB.columns[0]] + [col + "_y" for col in dfB.columns[1:]]

dfC = pd.merge(dfA, dfB, on='image_id')
print(dfC.head())


# Define the labels for your classification problem
labels = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']

# Create a list to store the accuracy for each label
label_acc = []
print("WIthout TTA")
# Loop through the labels and calculate accuracy, precision, and recall for each label
for label in labels:
    # Calculate accuracy for the label
    accuracy = (dfC[label] == dfC[label+'_y']).mean()
    label_acc.append(accuracy)
    
    # Calculate precision, recall, and F1-score for the label using classification_report
    report = classification_report(dfC[label], dfC[label+'_y'], output_dict=True)
    #print(report)
    #crt = report['1.0']['f1-score'] > report['macro avg']['f1-score'] ? 'macro avg' : '1.0'
    crt = '1.0'
    #precision = report[str(label)]['precision']
    #recall = report[str(label)]['recall']
    #f1_score = report[str(label)]['f1-score']
    
    # Print the metrics for the label
    print('Label:', label)
    print('Stats:',report[crt])
    print('---')


overall_acc=sum(label_acc)/clsz
print('overall_acc:', overall_acc)
dfC.to_csv('HAMdistE5E5withoutTTAandgroundtruth_alpha-0-35-Notemp.csv', index=False)